In [1]:
import pandas as pd
import os
import mysql.connector
from sqlalchemy import create_engine

In [2]:
# Create MySQL engine & MySQL connector
engine = create_engine('mysql+mysqlconnector://root@localhost:3306/cs6400_fa25_team58')
cnx = mysql.connector.connect(username='root', password='', host='127.0.0.1', database='cs6400_fa25_team58')

In [3]:
# Insert calendar data
add_calendar = '''
    INSERT INTO Calendar
    WITH RECURSIVE DateRange AS (
        SELECT CAST('2017-01-01' AS DATE) AS calendar_date
        UNION ALL
        SELECT DATE_ADD(calendar_date, INTERVAL 1 DAY) AS calendar_date 
        FROM DateRange
        WHERE calendar_date < '2019-12-31'
    )
    SELECT calendar_date FROM DateRange
    '''

with cnx.cursor() as cursor:
    cursor.execute(add_calendar)
    cnx.commit()

In [4]:
# Define file path
dirpath = os.path.join(os.path.dirname(os.getcwd()), 'Demo Data')
filepaths = [os.path.join(dirpath, filename) for filename in os.listdir(dirpath)]
filenames = [str.capitalize(filename[:-5]) for filename in os.listdir(dirpath)]

In [5]:
# Load data into dataframe
dfs_by_name = {fn: pd.read_csv(fp, delimiter='\t') for fn, fp in zip(filenames, filepaths)}

# List fields
for tbl, df in dfs_by_name.items():
    print(tbl + ": ", list(df.columns), end='\n\n')

Store:  ['store_number', 'phone', 'street_address', 'zip_code', 'city_name', 'city_state', 'city_population']

Sale:  ['date', 'store_number', 'product_id', 'quantity']

Membership:  ['date', 'store_number', 'membership_id', 'membership_type']

Holiday:  ['date', 'holiday_name']

Promotion:  ['date', 'product_id', 'discount_percent']

Manufacturer:  ['manufacturer_id', 'manufacturer_name', 'manufacturer_max_discount']

Product:  ['product_id', 'product_name', 'retail_price', 'manufacturer_id', 'category_name']



In [6]:
# Schema mapping
city_cols_mapping = {'city_name': 'city', 'city_state': 'state', 'city_population': 'population'}
dfs_by_name['City'] = dfs_by_name['Store'][city_cols_mapping.keys()].rename(columns=city_cols_mapping).drop_duplicates(ignore_index=True)
dfs_by_name['City']['city_id'] = range(1, len(dfs_by_name['City']) + 1)
dfs_by_name['Store'] = dfs_by_name['Store'] \
    .merge(dfs_by_name['City'], left_on=['city_name', 'city_state'], right_on=['city', 'state'], how='left') \
    [['store_number', 'phone', 'street_address', 'zip_code', 'city_id']]

membership_cols_mapping = {'date': 'signup_date', 'store_number': 'signup_store'}
dfs_by_name['Membership'] = dfs_by_name['Membership'].rename(columns=membership_cols_mapping)
dfs_by_name['MembershipType'] = dfs_by_name['Membership'][['membership_type']].rename(columns={'membership_type': 'type_name'}).drop_duplicates() 


dfs_by_name['Category'] = dfs_by_name['Product'][['category_name']].drop_duplicates(ignore_index=True)
dfs_by_name['ProductCategory'] = dfs_by_name['Product'][['product_id', 'category_name']]
dfs_by_name['Product'] = dfs_by_name['Product'].drop(columns=['category_name']).drop_duplicates(ignore_index=True)

dfs_by_name['Holiday'] = dfs_by_name['Holiday'].rename(columns={'date': 'holiday_date'})
dfs_by_name['Promotion'] = dfs_by_name['Promotion'].rename(columns={'date': 'promotion_date'})

dfs_by_name['Sale'] = dfs_by_name['Sale'].rename(columns={'date': 'sale_date'})

# Custom order
load_tables = ['City', 'Store', 'MembershipType', 'Membership', 'Manufacturer', 'Product', 'Category', 'ProductCategory', 'Holiday', 'Promotion', 'Sale']
dfs_by_name = {key: dfs_by_name[key] for key in load_tables if key in dfs_by_name}

In [7]:
# Reset load status
load_status = {tbl: False for tbl in load_tables}

In [8]:
label_value = None

for tbl, df in dfs_by_name.items():
    if load_status[tbl] == False:            
        df.to_sql(name=tbl, con=engine, if_exists='append', index=False)
        load_status[tbl] = True
    else:
        continue

In [9]:
load_status

{'City': True,
 'Store': True,
 'MembershipType': True,
 'Membership': True,
 'Manufacturer': True,
 'Product': True,
 'Category': True,
 'ProductCategory': True,
 'Holiday': True,
 'Promotion': True,
 'Sale': True}

In [10]:
cnx.close()